In [6]:
import pandas as pd
import numpy as np
import math
from decimal import Decimal
from datetime import timedelta
import time
import matplotlib.pyplot as plt

In [2]:
df_ori = pd.read_parquet('ekubo_market_depth_dataset.parquet')

In [3]:
df = df_ori.copy()

# Compute timestamp:
df['datetime'] = pd.to_datetime(df_ori['BLOCK_TIMESTAMP'])
df['timestamp'] = df['datetime'].astype('int64') // 10**9  # Convert nanoseconds to seconds
df = df.sort_values(by='timestamp')
df = df.reset_index(drop=True)

# Obtain names of Token0 and Token1:

# Tag different token address:
# Token0
# ETH: 0x049d36570d4e46f48e99674bd3fcc84644ddd6b96f7c741b1562b82f9e004dc7
# USDC: 0x053c91253bc9682c04929ca02ed00b3e423f6710d2ee7e0d5ebb06f3ecf368a8
# STRK: 0x04718f5a0fc34cc1af16a1cdee98ffb20c31f5cd61d6ab07201858f4287c938d

# Token1
# ETH: 0x049d36570d4e46f48e99674bd3fcc84644ddd6b96f7c741b1562b82f9e004dc7
# USDC: 0x053c91253bc9682c04929ca02ed00b3e423f6710d2ee7e0d5ebb06f3ecf368a8
# USDT: 0x068f5c6a61780768455de69077e07e89787839bf8166decfbf92b645209c0fb8
df['Token0_name'] = ''
df['Token1_name'] = ''

address = '0x049d36570d4e46f48e99674bd3fcc84644ddd6b96f7c741b1562b82f9e004dc7'
df.loc[df['TOKEN0_ADDRESS'] == address, 'Token0_name'] = 'ETH'

address = '0x053c91253bc9682c04929ca02ed00b3e423f6710d2ee7e0d5ebb06f3ecf368a8'
df.loc[df['TOKEN0_ADDRESS'] == address, 'Token0_name'] = 'USDC'

address = '0x04718f5a0fc34cc1af16a1cdee98ffb20c31f5cd61d6ab07201858f4287c938d'
df.loc[df['TOKEN0_ADDRESS'] == address, 'Token0_name'] = 'STRK'

address = '0x049d36570d4e46f48e99674bd3fcc84644ddd6b96f7c741b1562b82f9e004dc7'
df.loc[df['TOKEN1_ADDRESS'] == address, 'Token1_name'] = 'ETH'

address = '0x053c91253bc9682c04929ca02ed00b3e423f6710d2ee7e0d5ebb06f3ecf368a8'
df.loc[df['TOKEN1_ADDRESS'] == address, 'Token1_name'] = 'USDC'

address = '0x068f5c6a61780768455de69077e07e89787839bf8166decfbf92b645209c0fb8'
df.loc[df['TOKEN1_ADDRESS'] == address, 'Token1_name'] = 'USDT'


# Convert each column to either int or float:
columns_int = ['BLOCK_NUMBER', 'TOKEN0_DECIMALS', 'TOKEN1_DECIMALS', 'TICK_SPACING']
df[columns_int] = df[columns_int].applymap(lambda x: int(x))

columns_float = ['TOKEN0_RAW_AMOUNT', 'TOKEN0_REAL_AMOUNT', 'TOKEN1_RAW_AMOUNT', 'TOKEN1_REAL_AMOUNT',
                 'FEE_TIER', 'LIQUIDITY_AMOUNT', 'LOWER_TICK', 'UPPER_TICK', 'SWAP_TICK']
df[columns_float] = df[columns_float].applymap(lambda x: float(x))

# Create a 'tag' column for 'token0_token1'
df['tag'] = df['Token0_name']+'_'+df['Token1_name']

In [8]:
def _generate_aggregation(df2_input, columns_agg, tag, agg_type):
    ''' This is the function to compute and obtain daily aggregations inforamtion. 
    '''
    # Seperate input df to 'mint', 'burn', and 'swap'
    df2_input_mint = df2_input[df2_input['EVENT_NAME']=='Mint']
    df2_input_burn = df2_input[df2_input['EVENT_NAME']=='Burn']
    df2_input_swap = df2_input[df2_input['EVENT_NAME']=='Swap']
    
    # Obtain general daily information
    df2_output = pd.DataFrame(columns=columns_agg, index=[0])
    
    df2_output['tag'] = df2_input['tag'].iloc[0]
    
    if agg_type=='daily':
        df2_output['date'] = pd.Timestamp(df2_input['datetime'].dt.date.iloc[0])
    elif agg_type=='hourly':
        df2_output['date'] = df2_input['datetime'].dt.round('60min').iloc[0]
    
    df2_output['timestamp'] = (df2_output['date'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
    
    df2_output['token0'] = df2_input['Token0_name'].iloc[0]
    df2_output['token1'] = df2_input['Token1_name'].iloc[0]
    
    df2_output['total_events_daily_mint'] = len(df2_input_mint)
    df2_output['total_events_daily_burn'] = len(df2_input_burn)
    df2_output['total_events_daily_swap'] = len(df2_input_swap)
    df2_output['total_events_daily'] = len(df2_input_mint) + len(df2_input_burn) + len(df2_input_swap)

    # Compute 'mint' and 'burn'
    if len(df2_input_mint)>0:
        df2_output_tmp_mint = _compute_daily_price_amount(df2_input_mint)
        if len(df2_output_tmp_mint)>0:
            df2_output['token0_daily_price_median_mint']  = df2_output_tmp_mint['price_token0_daily'].iloc[0]
            df2_output['token0_daily_amount_mint'] = df2_output_tmp_mint['amount_token0_daily'].iloc[0]
            df2_output['liquidity_daily_sum_mint'] = df2_output_tmp_mint['liquidity_daily_sum'].iloc[0]
        else:
            df2_output['token0_daily_price_median_mint']  = np.nan
            df2_output['token0_daily_amount_mint'] = 0
            df2_output['liquidity_daily_sum_mint'] = 0            
    else:
        df2_output['token0_daily_price_median_mint']  = np.nan
        df2_output['token0_daily_amount_mint'] = 0
        df2_output['liquidity_daily_sum_mint'] = 0

    if len(df2_input_burn)>0:
        df2_output_tmp_burn = _compute_daily_price_amount(df2_input_burn)
        if len(df2_output_tmp_burn)>0:
            df2_output['token0_daily_price_median_burn']  = df2_output_tmp_burn['price_token0_daily'].iloc[0]
            df2_output['token0_daily_amount_burn'] = df2_output_tmp_burn['amount_token0_daily'].iloc[0]
            df2_output['liquidity_daily_sum_burn'] = df2_output_tmp_burn['liquidity_daily_sum'].iloc[0]
        else:
            df2_output['token0_daily_price_median_burn']  = np.nan
            df2_output['token0_daily_amount_burn'] = 0
            df2_output['liquidity_daily_sum_burn'] = 0            
    else:
        df2_output['token0_daily_price_median_burn']  = np.nan
        df2_output['token0_daily_amount_burn'] = 0
        df2_output['liquidity_daily_sum_burn'] = 0

    df2_output['token0_daily_amount_net_mint_burn'] = df2_output['token0_daily_amount_mint'] + df2_output['token0_daily_amount_burn']
    df2_output['liquidity_daily_net_mint_burn'] = df2_output['liquidity_daily_sum_mint'] + df2_output['liquidity_daily_sum_burn']
        
    # Compute 'swap':
    if len(df2_input_swap)>0:
        df2_output_tmp_swap = _compute_daily_price_amount(df2_input_swap)
        if len(df2_output_tmp_swap)>0:
            df2_output['token0_daily_price_median_swap'] = abs(df2_output_tmp_swap['ratio_token1_token0'].median())
            df2_output['token0_daily_price_min_swap']  = abs(df2_output_tmp_swap['ratio_token1_token0'].min())
            df2_output['token0_daily_price_max_swap']  = abs(df2_output_tmp_swap['ratio_token1_token0'].max())
            df2_output['token0_daily_price_std_swap']  = abs(df2_output_tmp_swap['ratio_token1_token0'].std())
        
            df2_output['token0_daily_amount_buy_swap']  = df2_output_tmp_swap['token0_daily_swap_buy'].iloc[0]
            df2_output['token0_daily_amount_sell_swap'] = df2_output_tmp_swap['token0_daily_swap_sell'].iloc[0]
            df2_output['token0_daily_amount_net_swap']  = df2_output['token0_daily_amount_buy_swap'] + df2_output['token0_daily_amount_sell_swap']
            # df_new['token0_daily_volumn_buy_swap_inUSD'] = df_new['token0_daily_price_median_swap'] * df_new['token0_daily_amount_buy_swap']
            # df_new['token0_daily_volumn_sell_swap_inUSD'] = df_new['token0_daily_price_median_swap'] * df_new['token0_daily_amount_sell_swap']
            # df_new['token0_daily_volumn_net_swap_inUSD'] = df_new['token0_daily_volumn_buy_swap_inUSD'] + df_new['token0_daily_volumn_net_swap_inUSD']
        else:
            df2_output['token0_daily_price_median_swap'] = np.nan
            df2_output['token0_daily_price_min_swap']  = np.nan
            df2_output['token0_daily_price_max_swap']  = np.nan
            df2_output['token0_daily_price_std_swap']  = np.nan
            df2_output['token0_daily_amount_buy_swap']  = np.nan
            df2_output['token0_daily_amount_sell_swap'] = np.nan
            df2_output['token0_daily_amount_net_swap']  = np.nan            
    else:
        df2_output['token0_daily_price_median_swap'] = np.nan
        df2_output['token0_daily_price_min_swap']  = np.nan
        df2_output['token0_daily_price_max_swap']  = np.nan
        df2_output['token0_daily_price_std_swap']  = np.nan
        df2_output['token0_daily_amount_buy_swap']  = np.nan
        df2_output['token0_daily_amount_sell_swap'] = np.nan
        df2_output['token0_daily_amount_net_swap']  = np.nan
    # Compute 'LIQUIDITY_AMOUNT' for 'Mint' and 'Burn' only:
    # df_new['liquidity_amount_sum'] = df_day['LIQUIDITY_AMOUNT'].sum()    
        
    df2_output = df2_output.sort_values(by='timestamp')
    df2_output = df2_output.reset_index(drop=True)
    return df2_output
    
###################################################################################################
def _compute_daily_price_amount(df_day_input):
    ''' To compute Token0 daily price and amount:
        Method: 
        Individual event price: 'TOKEN1_REAL_AMOUNT' / 'TOKEN0_REAL_AMOUNT'
        The final daily price '''
    df_day_output = pd.DataFrame()
    df_day_output['ratio_token1_token0'] = df_day_input['TOKEN1_REAL_AMOUNT']/df_day_input['TOKEN0_REAL_AMOUNT'] # Compute TOKEN1_REAL_AMOUNT/TOKEN0_REAL_AMOUNT
    df_day_output['TOKEN0_REAL_AMOUNT'] = df_day_input['TOKEN0_REAL_AMOUNT']     
    df_day_output = df_day_output[(df_day_output != 0) & (df_day_output != np.inf) & (df_day_output != -np.inf)].dropna() # Remove rows with 0, inf, or -inf values

    # Below only apply to 'mint' and 'burn'
    if (df_day_input['EVENT_NAME'].iloc[0]=='Mint') | (df_day_input['EVENT_NAME'].iloc[0]=='Burn'):
        # df_day_output['percentage'] = df_day_output['TOKEN0_REAL_AMOUNT']/df_day_output['TOKEN0_REAL_AMOUNT'].sum()
        # df_day_output['price_token0_daily'] = np.sum(df_day_output['ratio_token1_token0']*df_day_output['percentage'])
        df_day_output['price_token0_daily'] = df_day_output['ratio_token1_token0'].median()
        df_day_output['amount_token0_daily'] = df_day_output['TOKEN0_REAL_AMOUNT'].sum()
        df_day_output['liquidity_daily_sum'] = df_day_input['LIQUIDITY_AMOUNT'].sum()

    # Below only apply to 'swap'
    if df_day_input['EVENT_NAME'].iloc[0]=='Swap':
        df_day_output['token0_daily_swap_buy']   = df_day_input[df_day_input['TOKEN0_REAL_AMOUNT']>0]['TOKEN0_REAL_AMOUNT'].sum()
        df_day_output['token0_daily_swap_sell']  = df_day_input[df_day_input['TOKEN0_REAL_AMOUNT']<0]['TOKEN0_REAL_AMOUNT'].sum()
        df_day_output['token0_daily_swap_total'] = df_day_output['token0_daily_swap_buy'] + df_day_output['token0_daily_swap_sell']

    return df_day_output

###################################################################################################
def _generate_datetime_range(df2, agg_type):
    ''' Obatin a list with 'daily' or 'hourly' from the selected dataframe ''' 
    # Obtain the begin and end date of selected dataframe
    dt_begin = df2['datetime'].iloc[0]
    dt_end   = df2['datetime'].iloc[-1]
    d_begin = dt_begin.date()
    d_end   = dt_end.date() + timedelta(days=1)
    if agg_type=='daily':
        datetime_range = pd.date_range(start=d_begin, end=d_end)
    elif agg_type=='hourly':
        datetime_range = pd.date_range(start=d_begin, end=d_end, freq='H')
    return datetime_range
        
###################################################################################################
def create_aggregation_table(df, tags, agg_type):
    # Create an empty dataframe with defaulted column names 
    columns_agg = ['tag', 'token0', 'token1', 
                   'date', 'timestamp', 
                   
                   'token0_daily_price_median_mint', 'token0_daily_amount_mint', 
                   'token0_daily_price_median_burn', 'token0_daily_amount_burn', 
                   'token0_daily_amount_net_mint_burn',
                   
                   'liquidity_daily_sum_mint', 'liquidity_daily_sum_burn', 'liquidity_daily_net_mint_burn',
                   
                   'token0_daily_price_median_swap', 'token0_daily_price_min_swap', 
                   'token0_daily_price_max_swap', 'token0_daily_price_std_swap',
                   
                   'token0_daily_amount_buy_swap', 'token0_daily_amount_sell_swap', 'token0_daily_amount_net_swap',
                   # 'token0_daily_volumn_buy_swap_inUSD', 'token0_daily_volumn_sell_swap_inUSD', 'token0_daily_volumn_net_swap_inUSD',
                   
                   'total_events_daily_mint', 'total_events_daily_burn', 'total_events_daily_swap', 'total_events_daily' ]
    df_all = pd.DataFrame(columns=columns_agg)
    
    for tag in tags:
        print('Computing aggregation data for '+agg_type, tag)
        # Select data type: e.g., STRK_USDC_Mint, ETH_USDC_Burn, STRK_ETH_Swap, ... etc
        df2 = df[df['tag']==tag]
        df_agg_tmp = pd.DataFrame(columns=columns_agg)   # create a temporal dataframe

        # First, seperate into daily dataframe
        datetime_range = _generate_datetime_range(df2, agg_type)  # create either daily or hourly agg data
        
        for dt in datetime_range:
            if agg_type == 'daily':
                # Select data within a certain day
                df2_input = df2[df2['datetime'].dt.date == dt.date()] 
            elif agg_type == 'hourly':
                # Select data within a certain hour of a certain day
                df2_input = df2[ (df2['datetime'].dt.date == dt.date()) & (df2['datetime'].dt.hour == dt.hour) ]   
                
            if len(df2_input)>0:
                df_new = _generate_aggregation(df2_input, columns_agg, tag, agg_type)   # Compute aggregation values
                df_agg_tmp = pd.concat([df_agg_tmp, df_new])
                df_agg_tmp = df_agg_tmp.reset_index(drop=True)
    
        df_all = pd.concat([df_all, df_agg_tmp])
        
    df_all = df_all.sort_values(by='timestamp')
    df_all = df_all.reset_index(drop=True)
    
    df_all.to_csv('df_agg_'+agg_type+'_all.csv')
    print('End of Computing')

In [10]:
# Compute daily aggregation data
tags_daily = df['tag'].unique()
print(tags_daily)
start_time = time.time()
#######
create_aggregation_table(df, tags_daily, 'daily')
#######
end_time = time.time()
print("Total execution time:", round((end_time - start_time)/60, 2), "minutes")


# Compute hourly aggregation data
tags_hourly = ['STRK_ETH', 'STRK_USDC', 'STRK_USDT']
print(tags_hourly)  

start_time = time.time()
#######
create_aggregation_table(df, tags_hourly, 'hourly')
#######
end_time = time.time()
print("Total execution time:", round((end_time - start_time)/60, 2), "minutes")

['ETH_USDC' 'USDC_USDT' 'STRK_ETH' 'STRK_USDC' 'STRK_USDT']
Computing aggregation data for daily ETH_USDC
Computing aggregation data for daily USDC_USDT
Computing aggregation data for daily STRK_ETH
Computing aggregation data for daily STRK_USDC
Computing aggregation data for daily STRK_USDT
End of Computing
Total execution time: 2.63 minutes
['STRK_ETH', 'STRK_USDC', 'STRK_USDT']
Computing aggregation data for hourly STRK_ETH
Computing aggregation data for hourly STRK_USDC
Computing aggregation data for hourly STRK_USDT
End of Computing
Total execution time: 0.63 minutes


In [12]:
# Save individual files for ETH/USDC, USDC/USDT, STRK/ETH, STRK/USDC, STRK/USDT
for tag in tags_daily:
    df_daily = pd.read_csv('df_agg_daily_all.csv')
    dfn = df_daily[df_daily['tag']==tag]
    dfn.to_csv('df_agg_daily_'+tag+'.csv')
    print('Saved to ', 'df_agg_daily_'+tag+'.csv')

for tag in tags_hourly:
    df_hourly = pd.read_csv('df_agg_hourly_all.csv')
    dfn = df_hourly[df_hourly['tag']==tag]
    dfn.to_csv('df_agg_hourly_'+tag+'.csv')
    print('Saved to ', 'df_agg_hourly_'+tag+'.csv')

Saved to  df_agg_daily_ETH_USDC.csv
Saved to  df_agg_daily_USDC_USDT.csv
Saved to  df_agg_daily_STRK_ETH.csv
Saved to  df_agg_daily_STRK_USDC.csv
Saved to  df_agg_daily_STRK_USDT.csv
Saved to  df_agg_hourly_STRK_ETH.csv
Saved to  df_agg_hourly_STRK_USDC.csv
Saved to  df_agg_hourly_STRK_USDT.csv
